In [1]:
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
config_path = Path('../dash_config')

from omop_constructs.semantics import registry_engine


from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from orm_loader.helpers import Base  

from oa_cohorts.measurables import get_measurable_registry
from oa_cohorts.core import RuleTemporality, RuleTarget, RuleMatcher, ThresholdDirection, RuleCombination, ReportStatus
from oa_cohorts.query import (
    DashCohort, DashCohortDef, dash_cohort_def_map,
    Indicator, report_indicator_map,
    Measure, MeasureRelationship,
    MeasureNode, SubqueryNode, QueryNode, QueryPlan,
    Report, ReportCohortMap, ReportVersion,
    Subquery,
    QueryRule, ExactRule, HierarchyExclusionRule, HierarchyRule, AbsenceRule, ScalarRule, PhenotypeRule, SubstringRule,
    Phenotype, PhenotypeDefinition
)
import sqlalchemy.orm as so
import sqlalchemy as sa
from typing import Any, Iterable

Base.metadata.create_all(registry_engine)
Session = sessionmaker(bind=registry_engine, future=True)


def iter_leaf_subqueries(measure: Measure) -> Iterable["Subquery"]:
    """
    Yield all leaf Subquery objects used by this Measure tree.
    """
    if measure.subquery is not None:
        yield measure.subquery
        return

    for child in measure.children:
        yield from iter_leaf_subqueries(child)

def collect_report_subqueries(report: Report) -> list["Subquery"]:
    subqueries: list["Subquery"] = []

    # Indicator measures
    for m in report.indicator_measures:
        subqueries.extend(list(iter_leaf_subqueries(m)))

    # Cohort measures (from DashCohort definitions)
    for m in report.cohort_measures:
        subqueries.extend(list(iter_leaf_subqueries(m)))

    return subqueries

/Users/z3061723/Documents/CODE/OA_cohorts-1/.venv/lib/python3.13/site-packages/requests/__init__.py:113: RequestsDependencyWarning: urllib3 (2.6.3) or chardet (6.0.0.post1)/charset_normalizer (3.4.4) doesn't match a supported version!
  warnings.warn(


In [2]:
session = Session()

In [3]:
report = session.get(Report, 1)

In [4]:
sq = collect_report_subqueries(report)

In [5]:
from collections import Counter

def count_rule_targets_for_report(report: Report) -> Counter:
    subqueries = collect_report_subqueries(report)
    return Counter(sq.target for sq in subqueries)

In [6]:
seen = []
exemplars = []


for s in sq:
    if s.target not in seen:
        exemplars.append(s)
        seen.append(s.target)


In [ ]:
count_rule_targets_for_report(report)

In [7]:
len(exemplars)

14

In [8]:
exemplars[0]

Field,Value
ID,88
Name,Any treatment
Short name,any_tx
Target,tx_current_episode
Temporality,dt_treatment_start
Rule count,1
Field,Value
Type,presence


In [9]:
exemplars[1]

Field,Value
ID,85
Name,Curative RT
Short name,curative_rt
Target,intent_rt
Temporality,dt_rad
Rule count,1
Field,Value
Type,presence
Concept,Curative - procedure intent (4162591)


In [10]:
exemplars[2]

Field,Value
ID,86
Name,Any systemic therapy
Short name,systemic_tx
Target,tx_chemotherapy
Temporality,dt_treat
Rule count,1
Field,Value
Type,presence


In [11]:
exemplars[3]

Field,Value
ID,81
Name,Death
Short name,death
Target,demog_death
Temporality,dt_death
Rule count,1
Field,Value
Type,presence


In [12]:
exemplars[4]

<Subquery 'Stage I' target=dx_stage temporal=dt_stage>
  - <HierarchyRule id=4 hierarchy concept=1635838>

In [13]:
exemplars[5]

Field,Value
ID,123
Name,Bronchus cancer standard
Short name,bronch_stand
Target,dx_primary
Temporality,dt_current_start
Rule count,0


In [14]:
exemplars[6]

Field,Value
ID,100
Name,Pulmonary function
Short name,pulmonary_fn
Target,meas_concept
Temporality,dt_meas
Rule count,1
Field,Value
Type,exact


In [15]:
exemplars[7]

Field,Value
ID,103
Name,Palliative care referral
Short name,pall_care_ref
Target,obs_concept
Temporality,dt_obs
Rule count,1
Field,Value
Type,exact
Concept,Referral to palliative care service (4127745)


In [17]:
exemplars[8]

Field,Value
ID,145
Name,Lung Surgery
Short name,lung_surg
Target,tx_surgical
Temporality,dt_surg
Rule count,0


In [18]:
exemplars[9]

Field,Value
ID,102
Name,Discussed at MDT
Short name,mdt_discussion
Target,proc_concept
Temporality,dt_proc_start
Rule count,1
Field,Value
Type,exact
Concept,Assessment by multidisciplinary team (44790273)
Notes,Inconsistencies in observation / procedure domain?


In [19]:
exemplars[10]

Field,Value
ID,87
Name,Concurrent chemoRT
Short name,concurrent_chemort
Target,tx_concurrent
Temporality,dt_concurrent
Rule count,1
Field,Value
Type,presence


In [20]:
exemplars[11]

Field,Value
ID,128
Name,Treatment within 30 days of death
Short name,tx_30_days_dth
Target,tx_to_death_window
Temporality,dt_treatment_end
Rule count,0


In [22]:
exemplars[12]

Field,Value
ID,106
Name,GP referral to first specialist seen &lt; 14d
Short name,first_spec
Target,referral_to_specialist_window
Temporality,dt_consult
Rule count,2
Field,Value
Type,presence
Concept,Seen by clinical oncologist (4139715)
Notes,There is no suitable radiation oncologist code?


In [23]:
exemplars[13]

Field,Value
ID,109
Name,Mets to lung
Short name,lung_mets
Target,dx_mets
Temporality,dt_mets
Rule count,13
Field,Value
Type,exact
Concept,Metastasis to a different ipsilateral lobe of lung (36768171)
Field,Value
